In [1]:
import tensorflow as tf
import tensorflow.keras as kr

print(tf.__version__)
print(kr.__version__)

2024-05-14 10:14:51.247336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.1
2.11.0


初始化參數

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = './data/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
BEAM_SIZE = 8
PREDICT_LENGTH = 3
MAX_WORDS = 7500
EMBEDDING_WIDTH = 100


2024-05-24 12:04:04.604074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8')
text = file.read()
file.close()

text = text_to_word_sequence(text)
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i+WINDOW_LENGTH])
    targets.append(text[i+WINDOW_LENGTH])


轉換訓練樣本數據

In [3]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)
print(tokenizer.word_index)


X = np.array(fragments_indexed, dtype=np.int64)
y = np.zeros((len(targets_indexed), MAX_WORDS))
print(len(targets_indexed))
for i, targets_index in enumerate(targets_indexed):
    y[i, targets_index] = 1

    

{'UNK': 1, 'the': 2, 'and': 3, 'i': 4, 'of': 5, 'to': 6, 'my': 7, 'a': 8, 'in': 9, 'was': 10, 'that': 11, 'me': 12, 'with': 13, 'but': 14, 'had': 15, 'you': 16, 'he': 17, 'which': 18, 'it': 19, 'as': 20, 'not': 21, 'his': 22, 'for': 23, 'by': 24, 'on': 25, 'this': 26, 'from': 27, 'be': 28, 'have': 29, 'her': 30, 'at': 31, 'is': 32, 'when': 33, 'were': 34, 'or': 35, 'your': 36, 'she': 37, 'one': 38, 'an': 39, 'him': 40, 'all': 41, 'they': 42, 'so': 43, 'if': 44, 'will': 45, 'could': 46, 'are': 47, 'would': 48, 'been': 49, 'their': 50, 'we': 51, 'no': 52, 'who': 53, 'more': 54, 'these': 55, 'some': 56, 'should': 57, 'now': 58, 'yet': 59, 'before': 60, 'myself': 61, 'any': 62, 'what': 63, 'upon': 64, 'our': 65, 'do': 66, 'man': 67, 'them': 68, 'its': 69, 'only': 70, 'am': 71, 'into': 72, 'did': 73, 'life': 74, 'every': 75, 'than': 76, 'may': 77, 'then': 78, 'own': 79, 'first': 80, 'father': 81, 'shall': 82, 'might': 83, 'time': 84, 'can': 85, 'eyes': 86, 'said': 87, 'how': 88, 'such': 89,

製作模型並訓練

In [ ]:
training_model = Sequential()
training_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, input_length=None
))
training_model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
training_model.add(Dense(128, activation='relu'))
training_model.add(Dense(MAX_WORDS, activation='softmax'))
training_model.compile(loss='categorical_crossentropy', optimizer='adam')
training_model.summary()

hisotry = training_model.fit(X, y, validation_split=0.05,
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS, verbose=2, shuffle=True)


載入模型並做修改，使其可延續內部狀態

In [4]:

training_model = tf.keras.models.load_model('./model/6-1-autocomplete_2.keras')

# Check its architecture
training_model.summary()


2024-05-24 12:04:27.712286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         750000    
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 7500)              967500    
                                                                 
Total params: 1,982,844
Trainable params: 1,982,844
Non-trainable params: 0
_________________________________________________________________


In [5]:
inference_model = Sequential()
inference_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, batch_input_shape=(1, 1)
))
inference_model.add(LSTM(128, return_sequences=True, dropout=0.2, 
                        recurrent_dropout=0.2, stateful=True))
inference_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, stateful=True))
inference_model.add(Dense(128, activation='relu'))
inference_model.add(Dense(MAX_WORDS, activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)


Auto-Complete預測文字輸出

In [6]:
first_words = ['i', 'saw']
first_words_index = tokenizer.texts_to_sequences(first_words)
inference_model.reset_states()
predict_string = ''

for i, word_index in enumerate(first_words_index):
    x = np.zeros((1, 1), dtype=np.int64)
    x[0][0] = word_index[0]
    predict_string += first_words[i]
    predict_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
    
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts([[new_word_index]])
    x[0][0] = new_word_index
    predict_string += word[0]
    predict_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]

print(predict_string)
    
    
    

i saw the most man 


探索詞向量間相似程度

In [7]:
embeddings = training_model.layers[0].get_weights()[0]
lookup_words = ['the', 'saw', 'see', 'of', 'and', 'monster', 'frankenstein', 'read', 'eat']

for lookup_word in lookup_words:
    lookup_word_index = tokenizer.texts_to_sequences([lookup_word])
    print('words close to: ', lookup_word)
    lookup_embedding = embeddings[lookup_word_index[0]]
    word_indices = {}
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(embedding-lookup_embedding)
        word_indices[distance] = i
    # print(sorted(word_indices.keys())[:5])
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')
    

words close to:  the
the:  0.0
a:  1.2957003
an:  1.5401374
mischief:  1.7677956
disgusting:  1.7785587

words close to:  saw
saw:  0.0
child:  0.4977469
object:  0.53017384
assure:  0.5322889
rest:  0.538458

words close to:  see
see:  0.0
licence:  0.58005524
shaken:  0.58452773
tormented:  0.5941571
where:  0.5955395

words close to:  of
of:  0.0
in:  0.35789108
at:  0.39923733
by:  0.41019884
to:  0.42338255

words close to:  and
and:  0.0
are:  0.35406065
or:  0.3697343
be:  0.37196544
showers:  0.4020209

words close to:  monster
monster:  0.0
lieutenant:  0.4878582
effort:  0.5088524
rhine:  0.51656014
must:  0.5224606

words close to:  frankenstein
frankenstein:  0.0
2863:  0.5054841
been:  0.50797695
accordance:  0.5166158
f:  0.52113634

words close to:  read
read:  0.0
me:  0.4560609
mind:  0.52315605
philosophical:  0.5358197
say:  0.5376104

words close to:  eat
eat:  0.0
fail:  0.5669644
discipline:  0.5800065
flowing:  0.5985321
thanked:  0.60945565

